In [5]:
import os
import glob
import pandas as pd
from pathlib import Path
import sys
import subprocess


In [10]:
# set input path, models, output path, and other parameters
path_base = Path("/workspace/demo_image") # path to the directory including the images
path_CPout = "CPsegm_PSM"       # output directory name for cellpose
prefix_img_for_cellpose = "Series*_Processed001_ch01.tif"  # file name pattern
BF_or_FLUO = "FLUO"             # which image to use for cellpose

# output directory name for mmdetection
path_MMdet = "mmdet_results"
prefix_MMdet = "Series*_ch01"  # channel to use for mmdetection


# path to the mmdetection model and weights
model_path = Path("../model/PSM_exps_CorrectAnnotation_100percent/mask-rcnn_r50_fpn_2x_coco_100percent_nms.py")
weights_path = Path("../model/PSM_exps_CorrectAnnotation_100percent/epoch_24.pth")

# base_path 直下のディレクトリで、名前が 'Project' で始まるものをリストアップ
list_sample = [p.name for p in path_base.iterdir() if p.is_dir() and p.name.startswith('Project')]

print(list_sample)  # check the list of samples (strains) to analyze

['Project000_YSIY884', 'Project001_YSIY1448']


In [7]:
# Execute cellpose
dememseg_pybin = "/workspace/.venv_DeMemSeg/bin/python"
cellpose_pyscript = "../script/cellpose_segmentation.py"
for sample in list_sample:
    path_indir = path_base / sample
    !{dememseg_pybin} {cellpose_pyscript} {path_indir} {path_CPout} {prefix_img_for_cellpose} {BF_or_FLUO}

/workspace/demo_image/Project000_YSIY884
100%|█████████████████████████████████████████████| 1/1 [00:08<00:00,  8.75s/it]
=== /workspace/demo_image/Project000_YSIY884/Series015_480min_Lng_SVCC_Processed001.tif ===
100%|█████████████████████████████████████████| 168/168 [00:07<00:00, 22.50it/s]
=== /workspace/demo_image/Project000_YSIY884/Series015_480min_Lng_SVCC_Processed001_ch00.tif ===
100%|█████████████████████████████████████████| 168/168 [00:05<00:00, 29.05it/s]
=== /workspace/demo_image/Project000_YSIY884/Series015_480min_Lng_SVCC_Processed001_ch01.tif ===
100%|█████████████████████████████████████████| 168/168 [00:07<00:00, 22.20it/s]
/workspace/demo_image/Project001_YSIY1448
100%|█████████████████████████████████████████████| 1/1 [00:07<00:00,  7.05s/it]
=== /workspace/demo_image/Project001_YSIY1448/Series010_450min_Lng_SVCC_Processed001.tif ===
100%|█████████████████████████████████████████| 113/113 [00:04<00:00, 23.33it/s]
=== /workspace/demo_image/Project001_YSIY1448/Series

In [ ]:
dememseg_pybin = "/workspace/.venv_DeMemSeg/bin/python"

script_path = Path("/workspace/script")

if str(script_path) not in sys.path:
    sys.path.append(str(script_path))
    
import visualize_from_pred


# Execute mmdetection
mmdet_pyscript = "../script/mmdetection_psm.py"
for sample in list_sample:
    path_dir = path_base / sample / path_CPout
    list_indir = list(path_dir.rglob(f"crop_{prefix_MMdet}/bitwise_image"))
    
    for indir in list_indir:
        print(indir)

        subprocess.run([
            dememseg_pybin,
            mmdet_pyscript,
            str(indir),
            str(path_MMdet),
            str(model_path),
            str(weights_path)
        ])

        indir = Path(indir)
        outdir_pred = indir.parent / path_MMdet


        json_dir = Path(outdir_pred) / "preds"
        outdir_myvis = Path(outdir_pred) / "myvis"
        outdir_mypreds = Path(outdir_pred) / "mypreds"
        well_mask_dir = Path(outdir_pred).parent / 'mask_slices'
        score_thr = 0.5
        max_bbox_area = 80 * 80

        print(indir)
        print(outdir_pred)
        print(json_dir)
        print(outdir_myvis)
        print(outdir_mypreds)
        print(well_mask_dir)


        visualize_from_pred.apply_masks_and_save_all(image_dir=indir,
                                                    json_dir=json_dir,
                                                    well_mask_dir=well_mask_dir,
                                                    outdir_myvis=outdir_myvis,
                                                    outdir_mypreds=outdir_mypreds,
                                                    score_threshold=score_thr,
                                                    alpha=0.6,
                                                    min_area=0,
                                                    max_area=max_bbox_area)

/workspace/demo_image/Project000_YSIY884/CPsegm_PSM/crop_Series015_480min_Lng_SVCC_Processed001_ch01/bitwise_image
Loads checkpoint by local backend from path: ../model/PSM_exps_CorrectAnnotation_100percent/epoch_24.pth
05/21 04:58:16 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   

/workspace/.venv_DeMemSeg/lib/python3.9/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.TensorboardVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4 it/s  
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_100_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_100_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_101_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_10_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_10_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_10_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_112_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_114_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_117_0_crop_region_bitwise.png...
Processing crop_Series015_480min_Lng_SVCC_Processed001_ch01_117_0_crop_region_bitwise.png...
Processing 

Processing images: 100%|██████████| 168/168 [00:05<00:00, 30.02it/s]



✅ All overlays saved to: /workspace/demo_image/Project000_YSIY884/CPsegm_PSM/crop_Series015_480min_Lng_SVCC_Processed001_ch01/mmdet_results/myvis
✅ All filtered JSONs saved to: /workspace/demo_image/Project000_YSIY884/CPsegm_PSM/crop_Series015_480min_Lng_SVCC_Processed001_ch01/mmdet_results/mypreds
/workspace/demo_image/Project001_YSIY1448/CPsegm_PSM/crop_Series010_450min_Lng_SVCC_Processed001_ch01/bitwise_image
Loads checkpoint by local backend from path: ../model/PSM_exps_CorrectAnnotation_100percent/epoch_24.pth
05/21 05:00:04 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.
Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   

/workspace/.venv_DeMemSeg/lib/python3.9/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.TensorboardVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


Inference ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9 it/s  
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_102_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_102_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_102_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_103_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_103_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_103_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_110_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_110_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_110_0_crop_region_bitwise.png...
Processing crop_Series010_450min_Lng_SVCC_Processed001_ch01_110_0_crop_region_bitwise.png...
Processi

Processing images: 100%|██████████| 113/113 [00:03<00:00, 29.52it/s]


✅ All overlays saved to: /workspace/demo_image/Project001_YSIY1448/CPsegm_PSM/crop_Series010_450min_Lng_SVCC_Processed001_ch01/mmdet_results/myvis
✅ All filtered JSONs saved to: /workspace/demo_image/Project001_YSIY1448/CPsegm_PSM/crop_Series010_450min_Lng_SVCC_Processed001_ch01/mmdet_results/mypreds


In [14]:
# shape_masks.csvのパスリストの取得
list_indir = list(path_base.glob(f"Project*/{path_CPout}/crop_{prefix_MMdet}/{path_MMdet}/shape_masks.csv"))

# csvを結合する
combined_csv = pd.concat([pd.read_csv(f) for f in list_indir])

# ImageNameからmetadataを抽出
split_paths = combined_csv['JsonPath'].str.split('/', expand=True)
ProjectName = split_paths[3]
StrainName = split_paths[3].str.split('_', expand=True)[1]
TimeMin = split_paths[5].str.split('_', expand=True)[2]
Time = TimeMin.str.split('min', expand=True)[0]
SeriesName = split_paths[5].str.split('_', expand=True)[1]

# データフレームに一括でメタデータを追加
combined_csv['ProjectName'] = ProjectName
combined_csv['StrainName'] = StrainName
combined_csv['TimeMin'] = TimeMin
combined_csv['Time'] = Time
combined_csv['SeriesName'] = SeriesName

# 出力ファイルパスを生成して保存
output_path = path_base / f"{path_MMdet}.csv"
combined_csv.to_csv(output_path, index=False)

## Visualize PSM morphology by scatter plot
- YSIY884

- YSIY1448

In [1]:
import sys
sys.path.append("../script")

import scatter_plot_faceted
import importlib
importlib.reload(scatter_plot_faceted)

from scatter_plot_faceted import plot_each_strain_vs_wt_with_marginals

In [3]:
path_list = ["/workspace/demo_image/mmdet_results.csv"]
# 保存しない場合
plot = plot_each_strain_vs_wt_with_marginals(path_list, 
                                             show_plot=False,
                                             save = True,
                                             save_dir = "../figure/elongation_demo/")


/workspace/notebook/../script/scatter_plot_faceted.py:52: DtypeWarning: Columns (0,1,2,3,28,29,30,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wt = pd.read_csv(wt_path)


Saved: ../figure/elongation_demo/YSIY1448_vs_YSIY874_joint.png
Saved: ../figure/elongation_demo/YSIY884_vs_YSIY874_joint.png
